Imports

In [2]:
import sys
import os
import time
import re
import json
from urllib.parse import quote as urlencode
from urllib.request import urlretrieve

import http.client as httplib 

from astropy.table import Table,join
import numpy as np

import pprint
pp = pprint.PrettyPrinter(indent=4)
import SciServer
from SciServer import CasJobs, SkyQuery, SciDrive, SkyServer
import requests
import pandas as pd
import astropy as ap
import astropy.io.fits as astrofits
import astropy.io.votable as astrovot
import astropy.wcs as astrowcs
import astropy.units as u
import astropy.coordinates as astrocoords
import astropy.visualization as astrovis
import astropy.visualization.mpl_normalize as astromplnorm
import astropy.nddata as astronddata
import astropy.nddata.utils as astrondutils
import io
import pdb 
from IPython.core.display import display
from astropy.table import Table,vstack,Column
from astropy.utils.console import ProgressBar
from collections import deque
from pydoc import locate

import matplotlib.pyplot as mplplot

import gzip
import tarfile
import math
import shutil
from astropy.nddata.utils import Cutout2D

Definitions

In [3]:
#Query the MAST API
def mastQuery(request):
    server='mast.stsci.edu'

    # Grab Python Version 
    version = ".".join(map(str, sys.version_info[:3]))

    # Create Http Header Variables
    headers = {"Content-type": "application/x-www-form-urlencoded",
               "Accept": "text/plain",
               "User-agent":"python-requests/"+version}

    # Encoding the request as a json string
    requestString = json.dumps(request)
    requestString = urlencode(requestString)
    
    # opening the https connection
    conn = httplib.HTTPSConnection(server)

    executing = True
    while executing:
        # Making the query
        conn.request("POST", "/api/v0/invoke", "request="+requestString, headers)

        # Getting the response

        resp = conn.getresponse()
        head = resp.getheaders()
        content = resp.read().decode('utf-8')
        executing = 'EXECUTING' in content
        print('Still executing')
    # Close the https connection
    conn.close()

    return head,content


#Crossmatch between a catalog and the HSC catalog
def HscCrossmatch(data):
    crossmatchInput = {"fields":[{"name":"ra","type":"float"},
                                 {"name":"dec","type":"float"}],
                       "data":data}
    request = {"service":"Mast.Hsc.Crossmatch.MagAper2v3",
                   "data":crossmatchInput,
                   'params':{
                       "raColumn":"ra",
                       "decColumn":"dec",
                       'radius':0.0001,
                       'cache-breaker':10},
                   "pagesize":1000,
                   "page":1,
                   "format":"json",
                   "removecache":True}
        
    headers,outString = mastQuery(request)
    outData = json.loads(outString)
    
    return outData


#Get more information about the crossmatched galaxies
def getHSCMatches(matchId):
    request = {'service':'Mast.HscMatches.Db.v3',
               'params':{'input':matchId,
                         'cache-breaker':10},
               'format':'json',
               'page':1,
               'pagesize':4}   

    headers,outString = mastQuery(request)

    outData = json.loads(outString)

    return outData


#Hugh's code used to generate the correct url for downloading files from the HSC catalog
def genBundleRequest(imageNames, outfileNamePrefix='downloadBundle', extension='tar.gz'):
    baseUrl = "http://hla.stsci.edu/cgi-bin/getdata.cgi?config=ops&dataset="
    urlList = ",".join(['{}{}'.format(baseUrl, imageName) for imageName in imageNames])
    pathList = ['{}.fits'.format(imageName) for imageName in imageNames]

    request = {"service":"Mast.Bundle.Request",
               "params":{"urlList":urlList,
                         "filename":outfileNamePrefix,
                         "pathList":pathList,
                         "extension":extension,
                         'cache-breaker':10}
              }
    return request,outfileNamePrefix,extension


#Takes the output from the Bundle Request and downloads the files at that url
def downloadRequest(url):
    server='mast.stsci.edu'
    
    conn = httplib.HTTPSConnection(server)
    
    while True:
        conn.request("GET", url)
        resp = conn.getresponse()
        try:
            fileName = resp.getheader('Content-Disposition')[21:]
            fileContent = resp.read()
            with open(fileName,'wb') as FLE:
                FLE.write(fileContent)
            conn.close()

            return fileName
        except TypeError as e:
            print('Caught TypeError {}: {}, {}'.format(e, resp.getheader('Content-Disposition'), resp.read()))


    

#The following are with the help of Michael

#Reassign the datatypes from the json file to the correct datatype
def stringtype(typestr):
    typemap = {'string':str,'float':np.float64,'int':int,'boolean':bool,'date':str}
    return typemap[typestr]
    

#Create strings from values
def typecast(val,typestr):
    if val is None:
        return None
    elif typestr == 'string':
        return str(val)
    else:
        return locate(typestr)(val)
    

#Create astropy tables from the json files masking all values that create errors
def mastJson2Table(line):
    jsonObj = line
    if not jsonObj['data']:
        return None
    coldict = {field['name']:stringtype(field['type']) for field in jsonObj['fields']}
    keys,dtypes = zip(*coldict.items())
    
    rows = deque()
    
    for d in jsonObj['data']:
        row = [d[key] if d[key] is not None else '99' for key in keys]
        rows.append(row)
     
    table = Table(rows=list(rows),names=keys,dtype=dtypes)
    return table


#Stack tables together in one table
def read_matches(lines):
    tables = deque()
    for line in ProgressBar(lines):
        table = mastJson2Table(line)
        if table:
            tables.append(table)
    table = vstack(list(tables))
    return table



Catalogs

In [4]:
#SDSS Catalog
query = "select zoo2MainSpecz.dr8objid, Galaxy.ra, Galaxy.dec, Galaxy.petroR90_r, Galaxy.petroR90Err_r  FROM zoo2MainSpecz JOIN Galaxy ON zoo2MainSpecz.dr8objid = Galaxy.ObjID"
result = CasJobs.executeQuery(query, 'DR10', format='pandas')
ra_SDSS = result.ra.values
dec_SDSS = result.dec.values
obj = result.dr8objid.values
petroR90 = result.petroR90_r.values
petroR90Err_r = result.petroR90Err_r.values
display(result)

,dr8objid,ra,dec,petroR90_r,petroR90Err_r
0,1237651272972173326,172.392014,67.069298,9.490730,0.250569
1,1237651272972107811,172.093470,67.100123,4.603241,1.111299
2,1237651272971911352,171.113309,66.980701,9.458659,0.141786
3,1237651272971190298,166.778829,66.584584,10.897490,0.134753
4,1237651272959983754,125.666520,51.107838,8.469896,0.234417
5,1237651272959459604,124.565509,50.129684,9.621624,0.310719
6,1237651272959459410,124.433267,50.145042,17.791640,1.233928
7,1237651272959328300,124.221695,49.890164,9.699765,1.652695
8,1237651272957952290,121.469557,47.378339,8.540133,0.224804
9,1237651272972239030,172.893373,67.244998,6.367240,0.879745


In [29]:
print(obj[187799])

1237668494719975453


In [5]:
#HSC Crossmatche Catalog
bins = 500
size = int(len(ra_SDSS)/500)
lines = []
for i in range(int(size)):
    ra_dec_list = [{"ra":ras,"dec":decs} for ras,decs in zip(ra_SDSS,dec_SDSS)]
    data = ra_dec_list[i*bins:(i*bins)+bins]
    info = HscCrossmatch(data)
    #display(info)
    lines.append(info)
    if i%50 == 0:
        print('running')
stacked_table = read_matches(lines)
print(stacked_table['MatchRA'][0])


Still executing
running
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
running
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still ex

In [5]:
place = 2
print(stacked_table['dec'][place])
print(stacked_table['MatchDec'][place])

132.437019361891
132.43703985792766


In [6]:
coords_HSC = astrocoords.SkyCoord(ra=stacked_table['MatchRA'],dec=stacked_table['MatchDec'],unit=(u.deg,u.deg))
coords_SDSS = astrocoords.SkyCoord(ra_SDSS,dec_SDSS,unit=(u.deg,u.deg))

idx, d2d, d3d = coords_HSC.match_to_catalog_sky(coords_SDSS)
#Add things to table
objectid = Column(name='dr8objid', data=[obj[x] for x in idx])
radius = Column(name = 'radius',data = [petroR90[x] for x in idx])
error = Column(name = 'error',data = [petroR90Err_r[x] for x in idx])
stacked_table.add_column(objectid)
stacked_table.add_column(radius)

In [7]:
os.chdir('/home/idies/workspace/Storage/shaunowens/persistent/')
stacked_table.write('final.fits')
stacked_table.write('final.ecsv')

Download and Delete

In [8]:
filt = 'A_F475W'
matchIDs = [i['MatchID'] for i in stacked_table if isinstance(i[filt], np.float64) and i[filt] != 99.0 and not math.isnan(i[filt])]
images = [mastJson2Table(getHSCMatches(str(i)))['ImageName'] for i in matchIDs]

image_dict = {}
for i,name in enumerate(images):
    image_list = [j for j in name if 'f475w' in j]
    for k in image_list:
        image_dict[k] = [matchIDs[ind] for ind,out in enumerate(images) for In in out if In == k]

Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still executing
Still ex

In [10]:
os.chdir('/home/idies/workspace/Storage/shaunowens/persistent/Images')
name_list = []
for number,key in enumerate(image_dict):
    name_list.append(key)
query,filename,extension = genBundleRequest(name_list[:4])
pp.pprint(query)
print('1')
headers,bundleString = mastQuery(query)
bundleInfo = json.loads(bundleString)
pp.pprint(bundleInfo)
print('2')
downloadfile = downloadRequest(bundleInfo['url'])
tar = tarfile.open("downloadBundle.tar.gz")
os.chdir('/home/idies/workspace/persistent/Images/Images')
tar.extractall()
tar.close()
os.remove('System.String[]')
shutil.rmtree('downloadBundle')

{   'params': {   'cache-breaker': 10,
                  'extension': 'tar.gz',
                  'filename': 'downloadBundle',
                  'pathList': [   'hst_13777_03_acs_wfc_f475w.fits',
                                  'hst_10861_16_acs_wfc_f475w.fits',
                                  'hst_12918_04_acs_wfc_f475w.fits',
                                  'hst_9401_89_acs_wfc_f475w.fits'],
                  'urlList': 'http://hla.stsci.edu/cgi-bin/getdata.cgi?config=ops&dataset=hst_13777_03_acs_wfc_f475w,http://hla.stsci.edu/cgi-bin/getdata.cgi?config=ops&dataset=hst_10861_16_acs_wfc_f475w,http://hla.stsci.edu/cgi-bin/getdata.cgi?config=ops&dataset=hst_12918_04_acs_wfc_f475w,http://hla.stsci.edu/cgi-bin/getdata.cgi?config=ops&dataset=hst_9401_89_acs_wfc_f475w'},
    'service': 'Mast.Bundle.Request'}
1
Still executing
{   'bytesStreamed': 1642923086,
    'fileStatusList': {   'http://hla.stsci.edu/cgi-bin/getdata.cgi?config=ops&dataset=hst_10861_16_acs_wfc_f475w': '{"status":

KeyboardInterrupt: 

Make Cutouts

In [12]:
os.chdir('/home/idies/workspace/persistent/Images/Images')
for filename in os.listdir('/home/idies/workspace/persistent/Images/Images'):
    if filename == '.ipynb_checkpoints':
        continue
    print(filename)
    hdu = astrofits.open(filename)
    w = astrowcs.WCS(hdu[1].header)
    print(w)
    name = filename.replace('.fits', '')
    for number,z in enumerate(image_dict[name]):
        for i,j in enumerate(stacked_table['MatchID']):
            if j == z:
                ra_cut = stacked_table['ra'][i]
                dec_cut = stacked_table['dec'][i]
                coord = astrocoords.SkyCoord(ra_cut,dec_cut,unit=(u.deg,u.deg),frame='icrs')
                print('{:.10f}, {:.10f}'.format(ra_cut,dec_cut))
                cut = Cutout2D(hdu[1].data,position=coord,size=stacked_table['radius'][i]*u.arcsec,wcs=w)
                cut_fits = astrofits.PrimaryHDU(data=cut.data, header=cut.wcs.to_header())
                cut_fits.writeto('/home/idies/workspace/persistent/Images/Cuts/'+name+'_'+str(number)+'_cut.fits')

hst_10861_16_acs_wfc_f475w.fits
WCS Keywords

Number of WCS axes: 2
CTYPE : 'RA---TAN'  'DEC--TAN'  
CRVAL : 195.1006263592031  27.97542353872898  
CRPIX : 2542.499999999837  2513.999999999839  
CD1_1 CD1_2  : -1.3888888888888e-05  -2.4356242949037e-35  
CD2_1 CD2_2  : 0.0  1.38888888888896e-05  
NAXIS : 5084  5027
195.1165325959, 27.9559693891
195.0688286873, 27.9675072207
195.0753998947, 27.9565732905
195.1182936418, 27.9723909636
hst_12918_04_acs_wfc_f475w.fits
WCS Keywords

Number of WCS axes: 2
CTYPE : 'RA---TAN'  'DEC--TAN'  
CRVAL : 194.8907122352477  27.92233727634147  
CRPIX : 2838.999999999819  2820.999999999819  
CD1_1 CD1_2  : -1.3888888888888e-05  -4.8712485898075e-35  
CD2_1 CD2_2  : 0.0  1.38888888888896e-05  
NAXIS : 5677  5641
194.8748417585, 27.9564278137
194.9079503864, 27.9073190204
hst_9401_89_acs_wfc_f475w.fits
WCS Keywords

Number of WCS axes: 2
CTYPE : 'RA---TAN'  'DEC--TAN'  
CRVAL : 190.0604920944402  12.8736491606042  
CRPIX : 2518.499999999839  2508.49999999

In [26]:
os.chdir('/home/idies/workspace/Storage/shaunowens/persistent/Images')
name_list = []
for number,key in enumerate(image_dict):
    name_list.append(key)
bin_size = 5
for run in range(1+int(len(name_list)/bin_size)):
    query,filename,extension = genBundleRequest(name_list[run*bin_size:run*bin_size+bin_size])
    pp.pprint(query)
    headers,bundleString = mastQuery(query)
    bundleInfo = json.loads(bundleString)
    pp.pprint(bundleInfo)
    downloadfile = downloadRequest(bundleInfo['url'])
    tar = tarfile.open("downloadBundle.tar.gz")
    os.chdir('/home/idies/workspace/Storage/shaunowens/persistent/Images/Images')
    tar.extractall()
    tar.close()
    shutil.rmtree('downloadBundle')
    for name in name_list[run*bin_size:run*bin_size+bin_size]:
        if not os.path.isfile(name+'.fits'):
            os.rename('System.String[]',name+'.fits')


    for filename in os.listdir('/home/idies/workspace/Storage/shaunowens/persistent/Images/Images'):
        if filename.startswith('.'):
            continue
        if not os.path.isfile(filename):
            os.rename('System.String[]',filename)
        if filename.startswith('hst_12238_14_acs_wfc_f475w'):
            continue
        hdu = astrofits.open(filename)
        w = astrowcs.WCS(hdu[1].header)
        name = filename.replace('.fits', '')
        for number,z in enumerate(image_dict[name]):
            for i,j in enumerate(stacked_table['MatchID']):
                if j == z:
                    ra_cut = stacked_table['MatchRA'][i]
                    dec_cut = stacked_table['MatchDec'][i]
                    coord = astrocoords.SkyCoord(ra_cut,dec_cut,unit=(u.deg,u.deg),frame='icrs')
                    print('{:.10f}, {:.10f}'.format(ra_cut,dec_cut))
                    print(name)
                    cut = Cutout2D(hdu[1].data,position=coord,size=stacked_table['radius'][i]*u.arcsec,wcs=w)
                    cut_fits = astrofits.PrimaryHDU(data=cut.data, header=cut.wcs.to_header())
                    cut_fits.writeto('/home/idies/workspace/Storage/shaunowens/persistent/Images/Cuts/'+name+'_'+str(number)+'_cut.fits')
    for filename in os.listdir('/home/idies/workspace/Storage/shaunowens/persistent/Images/Images'):
        if filename.startswith('.'):
            continue
        os.remove(filename)
    os.chdir('/home/idies/workspace/Storage/shaunowens/persistent/Images')
    os.remove("downloadBundle.tar.gz")

{   'params': {   'cache-breaker': 10,
                  'extension': 'tar.gz',
                  'filename': 'downloadBundle',
                  'pathList': [   'hst_13324_11_acs_wfc_f475w.fits',
                                  'hst_9401_82_acs_wfc_f475w.fits',
                                  'hst_9401_08_acs_wfc_f475w.fits',
                                  'hst_9401_54_acs_wfc_f475w.fits',
                                  'hst_10594_03_acs_wfc_f475w.fits'],
                  'urlList': 'http://hla.stsci.edu/cgi-bin/getdata.cgi?config=ops&dataset=hst_13324_11_acs_wfc_f475w,http://hla.stsci.edu/cgi-bin/getdata.cgi?config=ops&dataset=hst_9401_82_acs_wfc_f475w,http://hla.stsci.edu/cgi-bin/getdata.cgi?config=ops&dataset=hst_9401_08_acs_wfc_f475w,http://hla.stsci.edu/cgi-bin/getdata.cgi?config=ops&dataset=hst_9401_54_acs_wfc_f475w,http://hla.stsci.edu/cgi-bin/getdata.cgi?config=ops&dataset=hst_10594_03_acs_wfc_f475w'},
    'service': 'Mast.Bundle.Request'}
Still executing
Still exec

In [ ]:
print(w)